## 1. Basic Capacity Expansion (without transmission & storage)
_Power System Optimization_

**Objective**: minimizing the cost of fixed and variable costs across a fleet of power generators to meet anticipated future electricity demand.  
**Note**:
Inter-temporal operating constraints (e.g. ramp limits), energy storage, network constraints and geospatial tradeoffs, network transmission, unit commitment decisions for thermal generators and operating reserves are ignored here

Increasing level of realism:  
1.1 Optimal thermal generator capacity expansion problem (Greenfield)  
1.2 Co-optimizing thermal generators and variable renewables  
1.3 Brownfield expansion and retirement decisions

... next levels: + storage units and power transmission

### 1.1 Optimal thermal generator capacity expansion problem (Greenfield)
**Problem Formulation**:  
Determine the most cost-effective mix of new thermal generation units for a power system, given projected electricity demand, fuel costs, and environmental constraints. Minimize the total cost of generation, which includes capital investment, operational, and maintenance costs, while satisfying operational constraints (such as generator limits). The "Greenfield" approach assumes no existing infrastructure, focussing solely on optimal new capacity decisions.

#### Basic formulation (without storage, renewable generators and transmission)
_see also: https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/03-Basic-Capacity-Expansion.ipynb_

$$G = \text{set of generators}$$  
$$H = \text{the set of hours of the year}$$

Objective Function:

$\begin{align}
   \min \sum_{g \in G} \left(FixedCost_g \times CAP_g + \sum_{h \in H} VarCost_g \times x_{g,h}\right)
    \quad\quad+ \sum_{h \in H} NSECost \times NSE_h
\end{align}$

subject to:
$\begin{align}
    & \sum_{g \in G} x_{g,h} + NSE_h = Demand_h & \forall \quad h \in H \\
    & x_{g,h} \leq CAP_g & \quad \forall g \in G \text{ , } h \in H \\
    & CAP_{g} \geq 0 & \quad \forall g \in G \\
    & NSE_{h} \geq 0 & \quad \forall  h \in H \\
    & x_{g,h} \geq 0 & \quad \forall  g \in G \text{ , } h \in H \\
\end{align}$

with:
$\begin{align}
    &FixedCost_g = Capex_g \times CRF_g + FixedOM_g \quad \forall g \in G\\
    &CRF_g = \frac{WACC_g(1+WACC_g)^{Life_g}}{(1+WACC_g)^{Life_g}-1} \quad \forall g \in G\\
    &VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g \quad \forall g \in G
\end{align}$

 

Decision Variables:  
$CAP_g$: the capacity (in MW) of each generation type  
$GEN_{g,h}$ : the generation (in MWh) produced by each generator in each hour  
$NSE_h$ :the quantity of involuntarily curtailed demand in each hour (in MWh)

_caution: demand is given in units of power (MW) and if resolution is not hourly, you need to multiply by the snapshot weighting (resolution) to get the energy consumed!_

... for 1.2 and 1.3 see [Power Systems Optimization Course](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/30c3463a375f27928430581cb416a5361cee31fa/Notebooks/03-Basic-Capacity-Expansion.ipynb)  

## 2. Single Node Capacity Expansion
_with PyPSA_

- the pypsa **single-node approach** focusses on optimizing the generation and storage capacities without modelling the transmission network here.
- for a detailed mathematical formulation of the capacity expansion problem considered by pypsa (with storage and renewable generators, etc) see: https://pypsa.readthedocs.io/en/latest/examples/capacity-expansion-planning-single-node.html 




### Loading technology-data from PyPSA database

In [ ]:
import pandas as pd

# assumptions and projections for 2030:
year = 2030
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{year}.csv"
costs = pd.read_csv(url, index_col=[0, 1])

# unit conversions
perkW_to_perMW = 1000
costs.loc[costs.unit.str.contains("/kW"), "value"] *= perkW_to_perMW
costs.unit = costs.unit.str.replace("/kW", "/MW")

# setting default values (for NaN)
default_values = {
    "FOM": 0,
    "VOM": 0,
    "efficiency": 1,
    "fuel": 0,
    "investment": 0,
    "lifetime": 25,
    "CO2 intensity": 0,
    "discount rate": 0.07,
}
costs = costs.value.unstack().fillna(default_values)

costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["OCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]
costs.at["CCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]

# calculating the short-term marginal generation costs (STMGC, €/MWh)
costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

# calculating capital costs
def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)
annuity = costs.apply(lambda x: annuity(x["discount rate"], x["lifetime"]), axis=1)
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

costs.head(3)

### Loading the time-series data for wind, solar and load (Germany 2015):

In [ ]:
GW_to_MW = 1000

url = "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
ts = pd.read_csv(url, index_col=0, parse_dates=True)

ts.load *= GW_to_MW
resolution = 4 #h
ts = ts.resample(f"{resolution}h").first()

ts.head(3)

### Setting-up the model with pypsa  (only one bus)

In [ ]:
# Tutorial: https://github.com/PyPSA/PyPSA/blob/9bc6f9142047a4285cfc99c2a5b71870e8cdbb83/examples/notebooks/capacity-expansion-planning-single-node.ipynb

import matplotlib.pyplot as plt

import pypsa

n = pypsa.Network()

n.add("Bus", name = "electricity-single-bus")
n.set_snapshots(ts.index)
n.snapshot_weightings.loc[:, :] = resolution 

# add multiple carriers/ technologies at once
technologies = [
    "onwind",
    "offwind",
    "solar",
    "OCGT",
    "hydrogen storage underground",
    "battery storage",
]
n.madd(
    "Carrier",
    technologies,
    color=["dodgerblue", "aquamarine", "gold", "indianred", "magenta", "yellowgreen"],
    co2_emissions=[costs.at[s, "CO2 intensity"] for s in technologies],
)
# add load time-series
n.add(
    "Load",
    "demand",
    bus="electricity-single-bus",
    p_set = ts.load,
)
n.loads_t.p_set.plot(figsize=(6, 2), ylabel="MW")

In [ ]:
# add one dispatchable generator to the one bus (open cycle gas turbine - OCGT)
# (offers sufficient and cheap enough backup capacity to run in periods of low wind and solar generation)
n.add(
    "Generator",
    "OCGT",
    bus="electricity-single-bus",
    carrier="OCGT",
    capital_cost=costs.at["OCGT", "capital_cost"],
    marginal_cost=costs.at["OCGT", "marginal_cost"],
    efficiency=costs.at["OCGT", "efficiency"],
    p_nom_extendable=True,
)

# add variable renewable generators with capacity factors to the one bus (p_max_pu)
for technology in ["onwind", "offwind", "solar"]:
    n.add(
        "Generator",
        technology,
        bus="electricity-single-bus",
        carrier=technology,
        p_max_pu=ts[technology],
        capital_cost=costs.at[technology, "capital_cost"],
        marginal_cost=costs.at[technology, "marginal_cost"],
        efficiency=costs.at[technology, "efficiency"],
        p_nom_extendable=True,
    )
n.generators_t.p_max_pu.loc["2015-03"].plot(figsize=(6, 2), ylabel="CF")

In [5]:
# add a battery storage with fixed energy-to-power ratio of 6h (if fully charged, battery can discharge at full capacity for 6 hours)
n.add(
    "StorageUnit",
    "battery storage",
    bus="electricity-single-bus",
    carrier="battery storage",
    max_hours=6,
    capital_cost=costs.at["battery inverter", "capital_cost"]
    + 6 * costs.at["battery storage", "capital_cost"],
    efficiency_store=costs.at["battery inverter", "efficiency"],
    efficiency_dispatch=costs.at["battery inverter", "efficiency"],
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
)

# add a hydrogen storage composed of an electrolysis to convert electricity to hydrogen, a fuel cell to re-convert hydrogen to electricity 
# and underground storage (e.g. in salt caverns). Assumed energy-to-power ratio of 168 hours, such that this type of storage can be used for weekly balancing.
capital_costs = (
    costs.at["electrolysis", "capital_cost"]
    + costs.at["fuel cell", "capital_cost"]
    + 168 * costs.at["hydrogen storage underground", "capital_cost"]
)

n.add(
    "StorageUnit",
    "hydrogen storage underground",
    bus="electricity-single-bus",
    carrier="hydrogen storage underground",
    max_hours=168,
    capital_cost=capital_costs,
    efficiency_store=costs.at["electrolysis", "efficiency"],
    efficiency_dispatch=costs.at["fuel cell", "efficiency"],
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
)

In [6]:
# add CO2-emission limit as global constraint to model a 100% renewable electricity system
n.add(
    "GlobalConstraint",
    "CO2Limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=0,
)

### Calculate the total system cost in Eur/year

In [ ]:
n.optimize(solver_name="highs")
n.objective

In [ ]:
# Optimized Capacities in MW
n.generators.p_nom_opt

In [ ]:
# Total energy generation by technology in MW
(n.statistics.capex() + n.statistics.opex()).div(1e3)

In [ ]:
n.storage_units.p_nom_opt  # MW

In [ ]:
# Total CO2-emmisions in t/h
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)
n.snapshot_weightings.generators @ emissions.sum(axis=1) # in t

In [ ]:
def plot_dispatch(n, time="2015-07"):
    p_by_carrier = n.generators_t.p.T.groupby(n.generators.carrier).sum().T.div(1e3)

    if not n.storage_units.empty:
        sto = n.storage_units_t.p.T.groupby(n.storage_units.carrier).sum().T.div(1e3)
        p_by_carrier = pd.concat([p_by_carrier, sto], axis=1)

    _, ax = plt.subplots(figsize=(6, 3))

    color = p_by_carrier.columns.map(n.carriers.color)

    p_by_carrier.where(p_by_carrier > 0).loc[time].plot.area(
        ax=ax,
        linewidth=0,
        color=color,
    )

    charge = p_by_carrier.where(p_by_carrier < 0).dropna(how="all", axis=1).loc[time]

    if not charge.empty:
        charge.plot.area(
            ax=ax,
            linewidth=0,
            color=charge.columns.map(n.carriers.color),
        )

    n.loads_t.p_set.sum(axis=1).loc[time].div(1e3).plot(ax=ax, c="k")

    plt.legend(loc=(1.05, 0))
    ax.set_ylabel("GW")
    ax.set_ylim(-200, 200)
plot_dispatch(n)

In [ ]:
def system_cost(n):
    tsc = n.statistics.capex() + n.statistics.opex()
    return tsc.droplevel(0).div(1e6)  # million €/a
system_cost(n).plot.pie(figsize=(2, 2))

In [ ]:
# compute the cost per unit of electricity consumed
demand = n.snapshot_weightings.generators @ n.loads_t.p_set.sum(axis=1)
system_cost(n).sum() * 1e6 / demand.sum()
#n.export_to_netcdf("network-new.nc")